# Brownian motion simulation
We want here to simulate a regular Brownian motion.
Then, we will try to simulate a fractional Brownian motion.
We also want to study the time required to make all these simulations.
## Importing packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from time import perf_counter
%matplotlib inline

## Some global simulation parameters

In [ ]:
N = 1000  # Nombre de points de discrétisation
times = []  # Dictionnaire des temps de calcul
time = np.linspace(10**-5, 10, N)  # Temps de discrétisation

## Simulating a regular Brownian motion

In [ ]:
plt.figure(figsize=(15, 9))
plt.title('Simulating regular Brownian Motion')

t1 = perf_counter()

for n in range(5):
    W = [0]
    for i in range(1,N):
        W.append(W[i-1] + np.random.normal(0, 1))
    plt.plot(time, W, label=f'W{n+1}')

t2 = perf_counter()
print(f'Elapsed time for N = {N}: {t2-t1} s')
times.append(t2-t1)
plt.legend()
plt.show()

## Simulating a fractional Brownian motion
Now, we are going to simulate a fractional Brownian motion.
For that, we know:
$$\operatorname{Cov}(X_t, X_s) = \frac12 \left(t^{2\alpha} + s^{2\alpha} - \left|t-s\right|^{2\alpha}\right)$$
where $\alpha$ is the Hurst parameter.
- If $\alpha=\frac 12$, then we have a regular Brownian motion.
- If $\alpha<\frac12$, then all increments are negatively correlated.
- If $\alpha>\frac12$, then all increments are positively correlated.

Then, we will use the Cholesky decomposition to simulate a fractional Brownian motion.

How ever, when we have exactly $t=s=0$, then the covariance is not defined, as have $\sigma^2_{W_0}=0$.
So we will start from $t=s=10^{-5}$.

In [ ]:
alphas = [0.3, 0.5, 0.8]  # Quelques valeurs de alpha
tmp = []
for alpha in alphas:
    plt.figure(figsize=(15, 9))
    plt.title(f'Simulating Fractionnal Brownian Motion with alpha = {alpha}')
    mat_cov = np.zeros((N, N))
    t1 = perf_counter()
    for t in range(N):
        for s in range(N):
            if (s > t):
                break
            mat_cov[t, s] = 0.5 * (time[t] ** (2 * alpha) + time[s] **
                                (2 * alpha) - np.abs(time[t] - time[s]) ** (2 * alpha))
            mat_cov[s,t] = mat_cov[t,s]

    C = np.linalg.cholesky(mat_cov)

    for i in range(5):
        Y = np.dot(C, np.random.normal(0, 1, N))
        Y[0] = 0
        plt.plot(time, Y, label=f'W{i+1}')
    
    t2 = perf_counter()
    
    print(f'Elapsed time for N = {N} and alpha = {alpha}: {t2-t1} s')
    tmp.append(t2-t1)
    plt.legend()
    plt.show()

times.append(np.mean(tmp))

## Riemann-Liouville Brownian motion

In [ ]:
tmp = []
for alpha in alphas:
    plt.figure(figsize=(15, 9))
    plt.title(f'Simulating Riemann-Liouville Brownian Motion with alpha = {alpha}')
    mat_cov = np.zeros((N, N))
    
    t1 = perf_counter()

    for t in range(N):
        for s in range(N):
            if (s > t):
                break
            mat_cov[t, s] = quad(lambda x: (time[t] - x) ** (alpha - 0.5) * (time[s] - x) ** (alpha - 0.5), 0, time[s])[0]
            mat_cov[s, t] = mat_cov[t, s]
    
    C = np.linalg.cholesky(mat_cov)

    for i in range(5):
        Y = np.dot(C, np.random.normal(0, 1, N))
        Y[0] = 0
        plt.plot(time, Y, label=f'W{i+1}')
        
    t2 = perf_counter()
    print(f'Elapsed time for N = {N} and alpha = {alpha}: {t2-t1} s')
    tmp.append(t2-t1)
    plt.legend()
    plt.show()

times.append(np.mean(tmp))
del tmp

We here notice that the last simulation takes a lot of time.

## Time required to simulate a Brownian motion
Here, we are going to plot the mean time required to simulate a Brownian motion of size $n=1000$.

In [ ]:
names = ['Regular Brownian Motion', 'Fractionnal Brownian Motion', 'Riemann-Liouville Brownian Motion']
plt.figure(figsize=(15, 9))
plt.title(f'Time of computation for N = {N}')
plt.bar(names, times)
plt.show()